In [2]:
# ---------------------------------------------------------------------------- #
# An implementation of https://arxiv.org/pdf/1512.03385.pdf                    #
# See section 4.2 for the model architecture on CIFAR-10                       #
# Some part of the code was referenced from below                              #
# https://github.com/pytorch/vision/blob/master/torchvision/models/resnet.py   #
# ---------------------------------------------------------------------------- #

import torch
import torchvision 
import torch.nn as nn
import torchvision.transforms as transforms

In [3]:
# Device config
device = ("cuda" if torch.cuda.is_available() else "cpu")

In [13]:
# Model parameters
batch_size = 100
num_epochs = 50
learning_rate = 0.001

In [10]:
# Image preprocessing modules
transform = transforms.Compose([
    transforms.Pad(4),
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32),
    transforms.ToTensor()
])

In [16]:
train_data = torchvision.datasets.CIFAR10(root = "../../data/",
                                       download=True,
                                       train=True,
                                       transform=transform)
test_data = torchvision.datasets.CIFAR10(root="../../data/",
                                      train=False,
                                      transform=transforms.ToTensor())

In [17]:
train_loader = torch.utils.data.DataLoader(dataset=train_data,
                                          shuffle=True,
                                          batch_size=batch_size)
test_loader = torch.utils.data.DataLoader(dataset=test_data,
                                         shuffle=True,
                                         batch_size=batch_size)

In [14]:
# 3x3 convolution
def conv3x3(input_channels, output_channels, stride=1):
    return(nn.Conv2d(input_channels, output_channels, kernel_size=3, stride=stride, padding=1, bias=False))

In [37]:
# Residual block
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(ResidualBlock, self).__init__()
        self.conv1 = conv3x3(in_channels, out_channels, stride)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)  #inplace=True means that it will modify the input directly, without allocating any additional output. It can sometimes slightly decrease the memory usage, but may not always be a valid operation (because the original input is destroyed). However, if you don’t see an error, it means that your use case is valid.
        self.conv2 = conv3x3(out_channels, out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = downsample
        
    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        if(self.downsample):
            residual = self.downsample(x)
        print("Downsample:", self.downsample)
        print("dim of out BEFORE adding residual:", out.size())
        out += residual
        print("dim of out AFTER adding residual:", out.size())
        out = self.relu(out)
        return(out)

In [65]:
# Resnet model
class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=10): # Layers = number residual blocks
        super(ResNet, self).__init__()
        self.in_channels = 16
        self.conv = conv3x3(3, 16)
        self.bn = nn.BatchNorm2d(16)
        self.relu = nn.ReLU(inplace=True)
        self.layer1 = self.make_layer(block, 16, layers[0])
        self.layer2 = self.make_layer(block, 32, layers[1], 2)
        self.layer3 = self.make_layer(block, 64, layers[2], 2)
        self.avg_pool = nn.AvgPool2d(8) # kernel_size = 8
        self.fc = nn.Linear(64, num_classes)
        
    def make_layer(self, block, out_channels, blocks, stride=1):
        downsample = None
        if (stride != 1) or (self.in_channels != out_channels):
            downsample = nn.Sequential(
            conv3x3(self.in_channels, out_channels, stride=stride),
            nn.BatchNorm2d(out_channels)
            )
        layers = []
        layers.append(block(self.in_channels, out_channels, stride, downsample))
        self.in_channels = out_channels
        for i in range(1, blocks):
            layers.append(block(out_channels, out_channels))
        return nn.Sequential(*layers) #Sequential layers of residual blocks
    
    def forward(self, x):
        out = self.conv(x)
        out = self.bn(out)
        out = self.relu(out)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.avg_pool(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return(out)

In [66]:
model = ResNet(ResidualBlock, [2, 2, 2]).to(device) # argList = [Block, Layers]; Layers = [2,2,2]

In [47]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [48]:
# For updating learning rate
def update_lr(optimizer, lr):
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr    

In [ ]:
# Train Model
curr_lr = learning_rate
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
    
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
    
        # Backward pass and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
        if (i+1) % 100 == 0:
            print ("Epoch [{}/{}], Step [{}/{}] Loss: {:.4f}".format(epoch+1, num_epochs, i+1, total_step, loss.item()))
            
    # Decay learning rate
    if(epoch+1) % 20 == 0:
        curr_lr /= 3
        update_lr(optimizer, curr_lr)

In [ ]:
# Test model
with torch.no_grad():
    total = correct = 0
    for i, (images, labels) in enumerate(test_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
        print('Accuracy of the model on the test images: {} %'.format(100 * correct / total))

In [ ]:
torch.save(model.state_dict(), "Resnet.ckpt")